In [28]:
import csv
import sqlite3
import pandas as pd
from datetime import datetime
from time import sleep


In [29]:
def remove_quotes(value):
    if isinstance(value, str):
        return value.replace('"', '')
    return value

In [30]:
file_path = 'C:\\dev\\Bank statement FI spending account.csv'
fi_spending = pd.read_csv(file_path, sep=';')

file_path = 'C:\\dev\\Bank statement FI savings account.csv'
fi_savings = pd.read_csv(file_path, sep=';')

file_path = 'C:\\dev\\Bank statement NL spending account.csv'
nl_spending = pd.read_csv(file_path, sep=',', converters={i: remove_quotes for i in range(100)})
# total_rows = len(nl_spending.index + fi_spending.index + fi_savings.index)
# display(total_rows)

In [31]:
# connecting to the database
connection = sqlite3.connect('C:\\dev\\financial_data.sqlite')
# cursor
cursor = connection.cursor()

#create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS banktransactions(
Id INTEGER PRIMARY KEY AUTOINCREMENT,
Date TEXT not null,
Amount DECIMAL(10,2) not null,
Message TEXT,
Category TEXT);
""")
connection.commit()

In [32]:
# insert data into table
for index, row in fi_spending.iterrows():
    # Convert each field to string, change comma to dot, and handle NaN values
    amount = str(row['Sum EURO']).replace(',', '.')
    receiver_payee = str(row['Receiver/Payee']).replace('"', '')
    explanation = str(row['Explanation']).replace('"', '')
    receiver_account_number = str(row['Receiver account number']) if not pd.isna(row['Receiver account number']) else ""
    message = str(row['Message']) if not pd.isna(row['Message']) else ""

    # Construct the full message string
    full_message = f"{receiver_payee}; {explanation}; {receiver_account_number}; {message}"
    # full_message = receiver_payee + "; " + explanation + "; " + receiver_account_number + "; " + message

    sql_query = f"""insert into banktransactions 
    (Date, Amount, Message) 
    values(
        "{datetime.strptime(row['Entry date'], '%d/%m/%Y').isoformat()}",
        {amount},
        "{full_message}")
    ;"""

    # print(sql_query)
    cursor.execute(sql_query)

connection.commit()

In [33]:
for index, row in fi_savings.iterrows():
    # Convert each field to string, change comma to dot, and handle NaN values
    amount = str(row['Sum EURO']).replace(',', '.')
    receiver_payee = str(row['Receiver/Payee']).replace('"', '')
    explanation = str(row['Explanation']).replace('"', '')
    receiver_account_number = str(row['Receiver account number']) if not pd.isna(row['Receiver account number']) else ""
    message = str(row['Message']) if not pd.isna(row['Message']) else ""

    # Construct the full message string
    full_message = f"{receiver_payee}; {explanation}; {receiver_account_number}; {message}"
    # full_message = receiver_payee + "; " + explanation + "; " + receiver_account_number + "; " + message

    sql_query = f"""insert into banktransactions 
    (Date, Amount, Message) 
    values(
        "{datetime.strptime(row['Entry date'], '%d/%m/%Y').isoformat()}",
        {amount},
        "{full_message}")
    ;"""

    cursor.execute(sql_query)

connection.commit()

In [34]:
for index, row in nl_spending.iterrows():
    # Convert each field to string, change comma to dot, and handle NaN values
    amount = str(row['amount'])
    description = str(row['description'])if pd.notna(row['description']) else ""

    sql_query = f"""insert into banktransactions 
    (Date, Amount, Message) 
    values(
        "{datetime.strptime(row['transactiondate'], '%Y%m%d').isoformat()}",
        {amount},
        "{description}")
    ;"""

    cursor.execute(sql_query)

connection.commit()

In [35]:
connection.close()